In [ ]:
import os
import random
import pickle
import wget 
from urllib.request import urlretrieve
from  urllib.request import urlopen
import urllib.error 
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt
import cv2
import imgaug as ia
from imgaug import augmenters as iaa
from enum import Enum, unique
import inspect
from PIL import Image
from tqdm import tqdm
from zipfile import ZipFile
from tensorflow.python.ops.variables import Variable
from functools import partial

%matplotlib inline

print('All modules imported.')

source = 'https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip'
repository = 'traffic-signs-data.zip'
batch_count = 128

In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy
'''
https://documen.tician.de/pycuda/tutorial.html
Note that you do not have to use pycuda.autoinit– initialization, context creation, and cleanup 
can also be performed manually, if desired.
'''


a_gpu = gpuarray.to_gpu(numpy.random.randn(4,4).astype(numpy.float32))
a_doubled = (2*a_gpu).get()
print(a_doubled)
print(a_gpu)



ImportError: /home/naaman/anaconda3/lib/python3.5/site-packages/pycuda/_driver.cpython-35m-x86_64-linux-gnu.so: undefined symbol: _ZNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEED1Ev

In [ ]:
from itertools import groupby

class Dataset(object):
    def __init__(self,source, repository):
        
        print('Status of Initialization Step #0')
        if source is None:
            print('None source parameter is not allowed. Please, provide a source parameter.')
            return None
        if len(source)==0:
            print("Empty source parameter is not allowed. Please, provide a valid source parameter.")
            return None
        if repository is None:
            print('None repository parameter is not allowed. Please, provide a repository parameter.')
            return None
        if len(repository)==0:
            print("Empty repository parameter is not allowed. Please, provide a valid repository parameter.")
            return None
        
        
        self.__source = source
        self.__repository=repository
        self.__datacached,self.__dataloaded = False,False
        self.__downloaded , self.__uncompressed = 0,0
        self.train_size, self.test_size,self.class_size,self.label_per_class = 0,0,0,[]
        self.image_shape = None
        self.max_size_feature = (0,0)
        
    def __wget(self, source, repository):
        try:
            print(source)
            content = urlopen(source)
            repository = open(source.split('/')[-1], 'w')
            repository.write(content.read())
            content.close()
            repository.close()
        except Exception as err:
            print('ERROR:', str(err))
            raise 
        
    
    def __download(self):
        print('Download Step #1')
        #print(self.__source)
        for src, rep in tqdm(zip(self.__source, self.__repository)):
            try :
                if not os.path.isfile(rep):
                    print('Start download files from '+src+'...')
                    urlretrieve(src,rep)
                    #rep = wget.download(src)
                    #self.__wget(src,rep)
                    print('Download finished.')
                    self.__downloaded += 1
                else :
                    print('Repository path '+rep+ ' is file! We can not save\
                            files because no directory path is provided.')
            except urllib.error.URLError: #Exception as inst : #
                    print('file not found corresponding to ' + src +'. moving on...')
                    raise
                    
        if self.__downloaded != 0 :
            print('Status of download Step : ' +str(self.__downloaded) + ' files are downloaded over '+\
                      str(len(self.__source)))
            
    def __uncompress(self):
        print('Uncompression Step #3')
        destfiles=[]
        for path in self.__repository :
        #Instantiate zipfile class as zipf context
            #print(path)
            with ZipFile(path,'r') as zipf:
                #extracting file name list using progressbar
                pbfiles = zipf.namelist()
                for fname in tqdm(pbfiles, unit='files'):
                    #check whether  file name do not point to a directory. Thus do not process it 
                    if not fname.endswith('/'):
                        #unzip the image file using the file name
                        print('Unzip the image file '+fname)
                        #with zipf.open(fname) as file:
                        zipf.extract(fname)

                        # Now store the uncompressed data
                        #fdest = fname[:-3]  # remove the '.gz' from the filename
                        #print(fdest)
                        destfiles.append(fname)
                        
            print('Status of uncompression Step :  '+ str(self.__uncompressed)+ \
                      'files are uncompressed over' +str(len(pbfiles)))
        destfiles = sorted(destfiles,key=os.path.getsize, reverse=True)
        return np.array(destfiles) 
    
    def __load(self,pkfile):
        # Reload the data
        print('Data Loading Step #4')
        train_features,train_labels = None,None
        if os.path.isfile(pkfile):
            #print(pkfile)
            with open(pkfile, 'rb') as f:
              pickle_data = pickle.load(f)
              features = pickle_data['features']
              labels = pickle_data['labels']
              del pickle_data  # Free up memory
            print('Status of Data loading from pickle file Step : successful')
            self.__dataloaded = True
        else :
            print('Status of Data loading from pickle file Step : failed')
        return features,labels
    
    def deserialize(self):
        desfiles = []
        print('The dataset deserialization process is starting...')
        
        if self.__downloaded==0:
            self.__download()
            
        if self.__uncompressed == 0:
            desfiles = self.__uncompress()
            
        if len(desfiles)==2 :
            train_features, train_labels = self.__load(desfiles[0])
            test_features,test_labels = self.__load(desfiles[1])
            self.train_size = train_features.shape[0]
            self.test_size = test_features.shape[0]
            self.image_shape = (train_features.shape[1],train_features.shape[2],train_features.shape[3])
            self.class_size = np.unique(train_labels).size
            self.label_per_class = np.array([(k,len(list(v))) for k,v in groupby(y_train)])
           
            for t in self.label_per_class:
                if(self.max_size_feature[1]<t[1]):
                    self.max_size_feature = t
            
            
        print('The dataset deserialization process is terminated...')
        
        return train_features,train_labels, test_features,test_labels
    
    def visualize(self, X_train, y_train) :
        index = random.randint(0, len(X_train))
        image = X_train[index].squeeze()

        plt.figure(figsize=(1,1))
        plt.imshow(image)
        print(y_train[index])
        
    def __max(self, X_train, y_train, augmenter):
        #Define the max(class,label/feature)
        self.max_size_feature = (0,0)
        for t in self.label_per_class:
            if(self.max_size_feature[1]<t[1]):
                self.max_size_feature = t
    
    def balance(self,X_train,y_train,augmenter,args):
        #for each (class,feature/label) different from max, 
        # augment(class, feature/label) by the amount of tuple = max(class,feature) - nb(class,feature)
        # Adjust(class, label)
        # Append(Augmented(class,feature), train_feature)
        #params = (1,1)
        #print((augmenter) (*params))
        augfeatures,auglabels =[],[]
        start=0
        for t in self.label_per_class:
            augfeatures.append(X_train[t[1]])
            auglabels.append(y_train[t[1]])
            if t[1]!=self.max_size_feature[1]:
                compensation = self.max_size_feature[1]-t[1]
                print(compensation)
                for i in range(compensation-1):
                    bf = X_train[start:start+t[1]]
                    bl = y_train[start:start+t[1]]
                    args = (bf,args[1])
                    ft = (augmenter)(*args)
                    augfeatures.append(ft)
                    auglabels.append(bl)
                    start += t[1]
                

In [ ]:
sources = [source]
repositories = [repository]

ds = Dataset(sources, repositories)
X_train, y_train,X_test, y_test = ds.deserialize()
print("Number of training examples =", ds.train_size)
print("Number of testing examples =", ds.test_size)
print("Image data shape =", ds.image_shape)
print("Number of classes =", ds.class_size)
#print("Number of label_per_class =", ds.label_per_class)
print("Max size of label_per_class =", ds.max_size_feature)




#ds.visualize(X_train, y_train)

In [ ]:
'''
1: Data Augmentation
    This technique provides a way genrerate more data from the existing and thus makes cnn or dnn to be 
    trained on huge number of training dataset to reach better performance and accuracy.
'''

class DataAugmenter(object):
    '''
    This library class is a data augmenter pipeline manager using the Augmenters Module referenced at this url:
    https://github.com/aleju/imgaug
   '''
    class Tasks(object):
        def __str__(self):
            return str(self.value)
        
        FlipVertical = 'flip_vertical'
        FlipHorizontal = 'flip_horizontal'
        Crop = 'crop'
        GaussianBlur = 'gaussian_blur'
        AdditiveGaussianNoise = 'additive_gaussian_noise'
        Dropout = 'dropout'
        Add = 'add'
        Multiply = 'multiply'
        ContrastNormalization = 'contrast_normalization'
        Affine = 'afine'
        ElasticTransformation = 'elastic_transformation'
        
        
    def __init__(self):
        
        # random example images
        #images = np.random.randint(0, 255, (16, 128, 128, 3), dtype=np.uint8)

        # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
        # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
        st = lambda aug: iaa.Sometimes(0.5, aug)
        self.pipeline = []
        # Define our sequence of augmentation steps that will be applied to every image
        # All augmenters with per_channel=0.5 will sample one value _per image_
        # in 50% of all cases. In all other cases they will sample new values
        # _per channel_.
        self.tasks = []
    
    def flip_horizontal(self):
        # horizontally flip 50% of all images
        if not "iaa.Fliplr" in self.pipeline:
            self.pipeline.append(iaa.Fliplr(0.5))
            
    def flip_vertical(self):
        # vertically flip 50% of all images
        if not "iaa.Flipud"  in self.pipeline:
            self.pipeline.append(iaa.Flipud(0.5))
            
    def crop(self):
        # crop images by crop range of 0-10%  of their height/width
        if not "iaa.Crop" in self.pipeline:
            self.pipeline.append(st(iaa.Crop(percent=(0, 0.1))))
    
    def gaussian_blur(self):
        # blur images with a sigma range between 0 and 3.0
        if not "iaa.GaussianBlur" in self.pipeline:
            self.pipeline.append(st(iaa.GaussianBlur((0, 3.0))))
    
    def additive_gaussian_noise(self):
        # add gaussian noise to images
        if "iaa.AdditiveGaussianNoise" in self.pipeline:
            self.pipeline.append(st(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.2), per_channel=0.5)))
    
    def dropout(self):
        # randomly remove up to 10% of the pixels  
        if not "iaa.Dropout" in self.pipeline:
            self.pipeline.append(st(iaa.Dropout((0.0, 0.1), per_channel=0.5)))
    
    def add(self):
        # change brightness of images (by -10 to 10 of original value)
        if not "iaa.Add" in self.pipeline:
            self.pipeline.append(st(iaa.Add((-10, 10), per_channel=0.5)))
    
    def multiply(self):
        # change brightness of images (50-150% of original value)
        if not "iaa.Multiply" in self.pipeline:
            self.pipeline.append(st(iaa.Multiply((0.5, 1.5), per_channel=0.5)))
        
    def contrast_normalization(self):
        # improve or worsen the contrast
        if not "iaa.ContrastNormalization" in self.pipeline:
            self.pipeline.append(st(iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5)))
    
    def affine(self):
        
        if not "iaa.Affine" in self.pipeline:
            self.pipeline.append(st(iaa.Affine(
                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
                    translate_px={"x": (-16, 16), "y": (-16, 16)}, # translate by -16 to +16 pixels (per axis)
                    rotate=(-45, 45), # rotate by -45 to +45 degrees
                    shear=(-16, 16), # shear by -16 to +16 degrees
                    order=ia.ALL, # use any of scikit-image's interpolation methods
                    cval=(0, 1.0), # if mode is constant, use a cval between 0 and 1.0
                    mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                )))
            
    def elastic_transformation(self):
        # apply elastic transformations with random strengths
        if not "iaa.ElasticTransformation" in self.pipeline:
            self.pipeline.append(st(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)))
    
        
    def register(self,tasks):
        #Allow to register the tailored data augmentation pipeline        
        print('Augmenter Pipeline registration process is starting ...')
        if tasks is None:
            print('None tasks parameter is not allowed. Please, provide a tasks parameter.')
            return None
        if len(tasks)==0:
            print("Empty tasks parameter is not allowed. Please, provide a valid tasks parameter.")
            return None
        
        self.tasks = tasks
        
        methnames = [m for m in dir(self) if inspect.ismethod(getattr(self, m))]                     

        for taskname in tasks :                         
            if taskname not in methnames:
                print('The following method does not exists: ', taskname)
                return
        for taskname in tasks :
            self.__getattribute__(taskname)()
        
        print('Augmenter Pipeline registration process is terminated ...')
    
    def execute(self, images, show=False):
        print('Augmenter Pipeline execution process is starting ...')
        pline = iaa.Sequential(self.pipeline, 
                             random_order = True # do all of the above in random order
                                )                         
        augmented_images = pline.augment_images(images)
        
        if show :
            # show an image with 8*8 augmented versions of image 0
            seq.show_grid(images[0], cols=8, rows=8)

        print('Augmenter Pipeline execution process is terminated ...')
        
        return augmented_images

def apply_augmentation2(batch_features, apply=False) :
    
    da = DataAugmenter()
    
    tasks = [da.Tasks.FlipHorizontal,da.Tasks.FlipVertical,da.Tasks.Crop,
                da.Tasks.GaussianBlur,da.Tasks.Dropout,
             da.Tasks.Affine,da.Tasks.ElasticTransformation]
    
    da.register(tasks)
    
    #batch_features, batch_labels  = load_batch(0,X_train, y_train)
    
    if apply :
        images_aug = da.execute(batch_features)
    else:
        images_aug = batch_features
        print('Not Apply')
    #train_on_images(images_aug)
    '''
    for idx,image in enumerate(images_aug):
        plt.figure(figsize=(1,1))
        plt.imshow(image)
        #print(batch_labels[idx])
    '''
    
    print(len(images_aug))
    
    return images_aug

def apply_augmentation(X_train, y_train, apply=False) :
    
    da = DataAugmenter()
    
    tasks = [da.Tasks.FlipHorizontal,da.Tasks.FlipVertical,da.Tasks.Crop,
                da.Tasks.GaussianBlur,da.Tasks.Dropout,
             da.Tasks.Affine,da.Tasks.ElasticTransformation]
    
    da.register(tasks)
    
    batch_features, batch_labels  = load_batch(0,X_train, y_train)
    
    if apply :
        images_aug = da.execute(batch_features)
    else:
        images_aug = batch_features
        print('Not Apply')
    #train_on_images(images_aug)
    
    for idx,image in enumerate(images_aug):
        plt.figure(figsize=(1,1))
        plt.imshow(image)
        print(batch_labels[idx])
        
def load_batch(batch_i,train_features,train_labels):
    batch_size = 1000 #train_features/batch_count
    # The training cycle
    #for batch_i in range(batch_count):
    # Get a batch of training features and labels
    batch_start = batch_i*batch_size
    batch_features = train_features[batch_start:batch_start + batch_size]
    batch_labels = train_labels[batch_start:batch_start + batch_size]
    return batch_features, batch_labels   
        

In [ ]:
#apply_augmentation(X_train, y_train,True) 
args = (None,True)
ds.balance(X_train,y_train,partial(apply_augmentation2),args)

In [ ]:
class DataPreProcessor(object):
    '''
    http://cs231n.github.io/neural-networks-2/#datapre
    There are three common forms of data preprocessing a data matrix X, 
    where we will assume that X is of size [N x D] (N is the number of data, D is their dimensionality).
    '''
    
    class Tasks(object):
        SimpleRescale = '_simple_rescale'
        MeanSubstract = '_mean_substract'
        FeatureStandardization = '_feature_standardization'
        
    class Normalizer(object):
        
        def _simple_rescale(self,images):
            images /= 255
            return images
        
        def _mean_substract(self,images):
            print('_zero_centred')
            print(images.dtype)
            #np.mean(images, axis = 0,dtype=np.uint8)
            images -= np.mean(images, axis = 0,dtype=np.uint8)
            return images
        
        def _feature_standardization(self, images):
            print('_normalize')
            images /= np.std(images, axis = 0)
            return images
        
    class Sequential(object):
        
        def __init__(self, sequence):
            self.sequence = sequence
        
        def preprocess_images(self,images):
            obj = self.sequence[0]
            self.sequence.remove(obj)
            
            for seq in self.sequence :
                images = getattr(obj, seq)(images)
                
            return images
    
    def __init__(self):
        self.pipeline = []
        ppc = self.Normalizer()
        self.pipeline.append(ppc)
    
    def simple_rescale(self):
        if not "_simple_rescale" in  self.pipeline:
            self.pipeline.append("_simple_rescale") 
    
    def mean_substract(self):
        if not "_mean_substract" in  self.pipeline:
            self.pipeline.append("_mean_substract") 
        
    def feature_standardization(self):
        if "_feature_standardization" in  self.pipeline :
            self.pipeline.append("_feature_standardization") 
        
    def register(self,tasks):
        print('Preprocessor Pipeline registration process is starting ...')
        if tasks is None:
            print('None tasks parameter is not allowed. Please, provide a tasks parameter.')
            return None
        if len(tasks)==0:
            print("Empty tasks parameter is not allowed. Please, provide a valid tasks parameter.")
            return None
        
        self.tasks = tasks
        
        methnames = [m for m in dir(self) if inspect.ismethod(getattr(self, m))]                     
        print(methnames)
        print(tasks)
        for taskname in tasks :                         
            if taskname not in methnames:
                print('The following method does not exists: ', taskname)
                return
        for taskname in tasks :
            self.__getattribute__(taskname)()
        
        print('Preprocessor Pipeline registration process is terminated ...')
        
    def execute(self, images, show=False):
        print('Preprocessor Pipeline execution process is starting ...')
        pline = self.Sequential(self.pipeline)
        
        imgs = pline.preprocess_images(images)
        
        if show :
            # show an image with 8*8 augmented versions of image 0
            seq.show_grid(imgs[0], cols=8, rows=8)

        print('Preprocessor Pipeline execution process is terminated ...')
        
        return imgs
    
def apply_preprocessing(X_train, y_train, apply=False) :
    dpp = DataPreProcessor()
    pipeline = [dpp.Tasks.MeanSubstract,dpp.Tasks.FeatureStandardization]
    dpp.register(pipeline)
    
    batch_features, batch_labels  = load_batch(0,X_train, y_train)
    
    if apply :
        images_aug = dpp.execute(batch_features)
    else:
        images_aug = batch_features
        print('Not Apply')
    #train_on_images(images_aug)
    
    for idx,image in enumerate(images_aug):
        plt.figure(figsize=(1,1))
        plt.imshow(image)
        print(batch_labels[idx])

In [ ]:

#apply_augmentation(X_train, y_train,False) 
#apply_preprocessing(X_train, y_train,True) 